In [49]:
import os
import json
import pandas as pd

results_dir = "../data/results/longmemeval_short_RAG_azure_gpt-5-mini_azure_gpt-5-mini"
json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]

results = []
for filename in json_files:
    filepath = os.path.join(results_dir, filename)
    with open(filepath, "r") as f:
        data = json.load(f)
        results.append(data)

df = pd.DataFrame(results)
df

,question_id,question,predicted_answer,answer,answer_is_correct
0,2ce6a0f2,How many different art-related events did I at...,Three.,4,False
1,gpt4_f49edff3,Which three events happened in the order from ...,First: helped your friend prepare the nursery ...,"First, I helped my friend prepare the nursery,...",True
2,618f13b2,How many times have I worn my new black Conver...,You’ve worn them six times.,six,True
3,0100672e,How much did I spend on each coffee mug for my...,"You spent $60 for 5 mugs, which is $60 ÷ 5 = $...",$12,True
4,70b3e69b,I was going through our previous conversation ...,Manolo García — the Spanish‑Catalan singer‑son...,Manolo García,True
5,88432d0a_abs,How many times did I bake egg tarts in the pas...,"Based on the provided conversation, there’s no...",The information provided is not enough. You di...,False
6,58ef2f1c,When did I volunteer at the local animal shelt...,You volunteered at the fundraising dinner on V...,February 14th,True
7,6613b389,How many months before my anniversary did Rach...,About two months before — roughly 2 months and...,2,True
8,ba61f0b9,How many women are on the team led by my forme...,Six women.,6,True
9,gpt4_e414231e,How many days passed between the day I fixed m...,0 days — both events were described as happeni...,4 days. 5 days (including the last day) is als...,False


In [50]:
print(df['answer_is_correct'].mean())

0.7692307692307693


In [51]:
for _, row in df.iterrows():
    answer = row['answer']
    predicted_answer = row['predicted_answer']
    correct = row['answer_is_correct']
    print(f"Question: {row['question']}")
    print(f"Answer: {answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Correct: {correct}")
    print("\n")


Question: How many different art-related events did I attend in the past month?
Answer: 4
Predicted Answer: Three.
Correct: False


Question: Which three events happened in the order from first to last: the day I helped my friend prepare the nursery, the day I helped my cousin pick out stuff for her baby shower, and the day I ordered a customized phone case for my friend's birthday?
Answer: First, I helped my friend prepare the nursery, then I helped my cousin pick out stuff for her baby shower, and lastly, I ordered a customized phone case for my friend's birthday.
Predicted Answer: First: helped your friend prepare the nursery (shopping at Buy Buy Baby).  
Second: helped your cousin pick out stuff for her baby shower (at Target).  
Third: ordered a customized phone case for your friend's birthday.
Correct: True


Question: How many times have I worn my new black Converse Chuck Taylor All Star sneakers?
Answer: six
Predicted Answer: You’ve worn them six times.
Correct: True


Question

In [52]:
with open("../data/longmemeval/longmemeval_s_cleaned.json", "r", encoding="utf-8") as f:
    longmemeval_df = pd.DataFrame(json.load(f)).reset_index(drop=True)

In [53]:
df.merge(longmemeval_df[["question_id", "question_type"]], on="question_id", how="inner").groupby(
    "question_type"
).agg(
    answer_is_correct_mean=("answer_is_correct", "mean"), count=("answer_is_correct", "count")
).sort_values(
    by="answer_is_correct_mean", ascending=False
).reset_index()

,question_type,answer_is_correct_mean,count
0,knowledge-update,1.000000,3
1,single-session-assistant,1.000000,1
2,single-session-user,1.000000,1
3,temporal-reasoning,0.666667,3
4,multi-session,0.600000,5
